<a href="https://colab.research.google.com/github/michelegaruti/Thesis-2021-Garuti/blob/main/Copy_of_EXIT_Thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import modules here
import pandas as pd
import numpy as np
import gdown
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [ ]:
#Time Aware layer for T_LSTM models:
'''
TLSTM_layer is obtained from the following Github Repository:

Nguyen, A., Chatterjee, S., Weinzierl, S., Schwinn, L., Matzner, M., & Eskofier, B.(2020).
Time matters: Time-aware lstms for predictive business processmonitoring; TLSTM_layer [Source Code]. Retrievable at:
https://github.com/annguy/time-aware-pbpm/blob/master/src/Models/TLSTM_layer.py. [Accessed on May 21st, 2021]

The MIT License (MIT)
Copyright (c) 2020, An Nguyen, Srijeet Chatterjee

Permission is hereby granted, free of charge, to any person obtaining a copy of 
this software and associated documentation files (the "Software"),
to deal in the Software without restriction, including without limitation the rights to use,
copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software,
and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies
or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED
TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE
OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

Link for detailed installation instructions of conda environment: https://github.com/annguy/time-aware-pbpm
'''
import tensorflow as tf
from tensorflow import keras
import numpy as np


class TLSTM_layer(keras.layers.Layer):
    """
    Time-ware LSTM inherits from tf.keras.layers.Layer. Implmentation is based on [1]
    [1] Baytas, Inci M., Cao Xiao, Xi Zhang, Fei Wang, Anil K. Jain, and Jiayu Zhou. “Patient Subtyping
    via Time-Aware LSTM Networks.”  KDD ’17
    Attributes
    ----------
    input_dim : int
        the dimension of the input layer.
    hidden_dim : int
        the dimension of the hidden layer.
    trainable : bool
        set the variable to be trainable or not.
    Wi, Wf, Wog : tensor
        weight matrices corresponding to the input, forget and output gates respectively,
        modifying the input connections.
    Ui, Uf, Uog : tensor
        weight matrices corresponding to the input, forget and output gates respectively,
        modifying the hidden connections.
    bi, bf, bog : tensor
        the bias vectors corresponding to the input, forget and output gates, respectively.
    Wc, Uc, bc : tensor
        the weight matrices and bias vector to compute the current cell memory.
    W_decomp, b_decomp : tensor
        subspace decompostion weight matrix and bias vector.
    """

    def __init__(self,
                 hidden_dim,
                 return_sequence=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 trainable=True,
                 supports_masking=True,
                 name=None):
        """Constructor of T-LSTM objects.
        Parameters
        ----------
        hidden_dim : int
            dimension of the hidden layer.
        return_sequence : bool
            weather to return all hidden states. Default: True
        kernel_initializer : str
            Initializer for the kernel weights matrix, used for the linear transformation of the inputs.
             Default: glorot_uniform.
         bias_initializer: str
             Initializer for the bias vector. Default: zeros.
        trainable :bool
            set the variables (weight and bias) to be trainable (for training) or not (for testing).
        supports_masking : bool
            weather layer should supporting masking
        name : str
            layers name
        """
        super(TLSTM_layer, self).__init__(name=name)
        # Signal that the layer is safe for mask propagation
        self.supports_masking = supports_masking
        # set initializer
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        # set dimensions
        self.hidden_dim = hidden_dim
        self.return_sequence = return_sequence
        self.trainable = trainable

    def build(self, input_shape):
        
        self.input_dim = input_shape[-1]
        # Set trainable weights
        self.Wi = self.add_weight(shape=(self.input_dim, self.hidden_dim),
                                  initializer=self.kernel_initializer,
                                  trainable=True,
                                  name='Input_Hidden_weight')
        self.Ui = self.add_weight(shape=(self.hidden_dim, self.hidden_dim),
                                  initializer=self.kernel_initializer,
                                  trainable=True,
                                  name='Input_State_weight')
        self.bi = self.add_weight(shape=self.hidden_dim,
                                  initializer=self.bias_initializer,
                                  trainable=True,
                                  name='Input_Hidden_bias')

        self.Wf = self.add_weight(shape=(self.input_dim, self.hidden_dim),
                                  initializer=self.kernel_initializer,
                                  trainable=True,
                                  name='Forget_Hidden_weight')
        self.Uf = self.add_weight(shape=(self.hidden_dim, self.hidden_dim),
                                  initializer=self.kernel_initializer,
                                  trainable=True,
                                  name='Forget_State_weight')
        self.bf = self.add_weight(shape=self.hidden_dim,
                                  initializer=self.bias_initializer,
                                  trainable=True,
                                  name='Forget_Hidden_bias')

        self.Wog = self.add_weight(shape=(self.input_dim, self.hidden_dim),
                                   initializer=self.kernel_initializer,
                                   trainable=True,
                                   name='Output_Hidden_weight')
        self.Uog = self.add_weight(shape=(self.hidden_dim, self.hidden_dim),
                                   initializer=self.kernel_initializer,
                                   trainable=True,
                                   name='Output_State_weight')
        self.bog = self.add_weight(shape=self.hidden_dim,
                                   initializer=self.bias_initializer,
                                   trainable=True,
                                   name='Output_Hidden_bias')

        self.Wc = self.add_weight(shape=(self.input_dim, self.hidden_dim),
                                  initializer=self.kernel_initializer,
                                  trainable=True,
                                  name='Cell_Hidden_weight')
        self.Uc = self.add_weight(shape=(self.hidden_dim, self.hidden_dim),
                                  initializer=self.kernel_initializer,
                                  trainable=True,
                                  name='Cell_State_weight')
        self.bc = self.add_weight(shape=self.hidden_dim,
                                  initializer=self.bias_initializer,
                                  trainable=True,
                                  name='Cell_Hidden_bias')

        self.W_decomp = self.add_weight(shape=(self.hidden_dim, self.hidden_dim),
                                        initializer=self.kernel_initializer,
                                        trainable=True,
                                        name='Decomposition_Hidden_weight')
        self.b_decomp = self.add_weight(shape=self.hidden_dim,
                                        initializer=self.bias_initializer,
                                        trainable=True,
                                        name='Decomposition_Hidden_bias_enc')

    def tlstm_unit(self, prev_hidden_memory, concat_input):
        """The update equations for the time-aware LSTM.
        Parameters
        ----------
        prev_hidden_memory : list of tensor
            a list containing the previous hidden states and previous cell memory (1, 2, batch_size, hidden_dim)
        concat_input : tensor
            a tensor containing all the input data and the elapsed times (batch_size, input_dim+1)
        Returns
        -------
        tensor
            Updated prev_hidden_memory and the cell state concatenated (1, 2, batch_size, hidden_dim)
        """
        prev_hidden_state, prev_cell = tf.unstack(prev_hidden_memory)

        batch_size = tf.shape(concat_input)[0]
        x = tf.slice(concat_input, [0, 1], [batch_size, self.input_dim])  # (batch_size, features)
        t = tf.slice(concat_input, [0, 0], [batch_size, 1])  # (batch_size, 1)

        # Dealing with time irregularity

        # Map elapse time in days or months
        T = self.map_elapse_time(t)

        # Decompose the previous cell if there is an elapse time
        C_ST = tf.nn.tanh(tf.matmul(prev_cell, self.W_decomp) + self.b_decomp)
        C_ST_dis = tf.multiply(T, C_ST)
        # if T is 0, then the weight is one
        prev_cell = prev_cell - C_ST + C_ST_dis

        # Input gate
        i = tf.sigmoid(tf.matmul(x, self.Wi) + tf.matmul(prev_hidden_state, self.Ui) + self.bi)

        # Forget Gate
        f = tf.sigmoid(tf.matmul(x, self.Wf) + tf.matmul(prev_hidden_state, self.Uf) + self.bf)

        # Output Gate
        o = tf.sigmoid(tf.matmul(x, self.Wog) + tf.matmul(prev_hidden_state, self.Uog) + self.bog)

        # Candidate Memory Cell
        C = tf.nn.tanh(tf.matmul(x, self.Wc) + tf.matmul(prev_hidden_state, self.Uc) + self.bc)

        # Current Memory cell
        Ct = f * prev_cell + i * C

        # Current Hidden state
        current_hidden_state = o * tf.nn.tanh(Ct)

        return tf.stack([current_hidden_state, Ct])

    def call(self, x, time):
        """ Compute the hidden states for the input sequence.
        Parameters
        ----------
        x : array
            the input of shape (batch_size, sequence_length, input_dimension).
        time : array
            the elapsed time of shape (batch_size, sequence_length).
        Returns
        -------
        tensor
            The computed hidden states of shape (batch_size, seq_len, hidden_dim)
        """
        batch_size = tf.shape(x)[0]
        scan_input_ = tf.transpose(x, perm=[2, 0, 1])
        scan_input = tf.transpose(scan_input_)  # scan_input: (seq_length, batch_size, input_dim)
        scan_time = tf.transpose(time)

        initial_hidden = tf.zeros([batch_size, self.hidden_dim], tf.float32)
        ini_state_cell = tf.stack([initial_hidden, initial_hidden])

        # Make scan_time have shape of (seq_length, batch_size, 1)
        scan_time = tf.reshape(scan_time, [tf.shape(scan_time)[0], tf.shape(scan_time)[1], 1])
        concat_input = tf.concat([scan_time, scan_input], 2)  # concat_input: (seq_length, batch_size, input_dim+1)

        # Call TLSTM_Unit recursively for each time step (seq_length, 2, batch_size, hidden_dim)
        packed_hidden_states = tf.scan(self.tlstm_unit, concat_input, initializer=ini_state_cell, name='states')

        # Extract all the hidden states
        all_states = packed_hidden_states[:, 0, :, :]  # (seq_length, batch_size, hidden_dim)
        all_states = tf.transpose(all_states, perm=[1, 0, 2])  # (batch_size, seq_len, hidden_dim)

        if self.return_sequence:
            return all_states  # return all hidden states
        else:
            return all_states[:, -1, :]  # return last hidden state

    def map_elapse_time(self, t):
        """ Given the time intervals, compute the decay terms.
        Parameters
        ----------
        t : tensor
            input time intervals.
        Returns
        -------
        tensor
            The computed decay terms.
        """
        c1 = tf.constant(1, dtype=tf.float32)
        c2 = tf.constant(2.7183, dtype=tf.float32)

        T = tf.math.divide(c1, tf.math.log(t + c2), name='Log_elapse_time')
        Ones = tf.ones([1, self.hidden_dim], dtype=tf.float32)
        T = tf.matmul(T, Ones)

        return T


    def get_config(self):
        config = ({'hidden_dim': self.hidden_dim,
                   'supports_masking': self.supports_masking,
                   'kernel_initializer': self.kernel_initializer,
                   'bias_initializer': self.bias_initializer,
                   'return_sequence': self.return_sequence,
                   'trainable': self.trainable,
                   'name': self.name})

        return config


if __name__ == "__main__":
    # %% run on dummy tensors
    inp_dim = 2
    hid_dim = 5

    time_steps = 10
    batchSize = 20

    x_ = np.ones([batchSize, time_steps, inp_dim])
    # leftpad sequences with zeros -> simulate variable length inputs in batch
    # first 2 sequences have 2 leading zeros, next two 4 leading zeros, ...
    x_[:2, :2, :] = 0
    x_[2:4, :4, :] = 0
    x_[4:6, :6, :] = 0
    x_ = tf.convert_to_tensor(x_, dtype='float32')
    dts = np.ones([batchSize, time_steps])
    dts[:2, :2] = 0
    dts[2:4, :4] = 0
    dts[4:6, :6] = 0
    dts = tf.convert_to_tensor(dts, dtype='float32')

    # initialize TLSTM layer object
    TLSTM_layer_ = TLSTM_layer(hid_dim, return_sequence=True)
    hidden = TLSTM_layer_(x_, dts)
    print(tf.shape(hidden))

    TLSTM_layer_ = TLSTM_layer(hid_dim, return_sequence=False)
    hidden = TLSTM_layer_(x_, dts)
    print(tf.shape(hidden))

In [ ]:
!pip install kora -q
from kora import drive
drive.link_nbs()
import import_ipynb
import src
'''!pip install scikit-learn==0.23.1
!pip install imbalanced-learn==0.7.0'''

In [ ]:
#Dataset
'''
The full dataset can be downloaded following this Github link:

Requena, B., Cassani, G., Tagliabue, J., Greco, C., & Lacasa, L. (2020).
Shopper intentprediction from clickstream e-commerce data with minimal browsing information.
https://github.com/coveooss/shopper-intent-prediction-nature-2020
'''

df = pd.read_csv('./release_10_23_2020.csv')

In [ ]:
 #DOWNLOADING DATASET AND SETTING UP NOTEBOOK

!gdown --id 1GU_VafffNuwPvlnJNnMcKlDnSvEhFLEY

df = pd.read_csv('./release_10_23_2020.csv')

In [ ]:
## Pre-processing function
def preprocess(df):

  # rename actions
  df['product_action'] = df['product_action'].replace([np.nan,'detail', 'add', 'remove', 'click', 'purchase',],[1,2,3,4,5,7])

  # make sequences
  df = df.groupby('session_id_hash')[['product_action', 'server_timestamp_epoch_ms']].agg(list).reset_index()

  #make labels
  df['label'] = np.where(df.product_action.map(set([7]).issubset), 0, 1)
  
  #trim sequences after first purchase event
  def trim(df):
    sequence_action = []
    timestamp = []
    for item, time in zip(df.product_action, df.server_timestamp_epoch_ms):
      if 7 in set(item):
        for e in range(len(item)):
          if item[e] == 7:
            item1 = item[: e]
            sequence_action.append(item1)
            time1 = time[: e] 
            timestamp.append(time1)
            break
      else:
        sequence_action.append(item)
        timestamp.append(time)
    df['sequence_action'] = sequence_action
    df['timestamp'] = timestamp
    return df
  trim(df)

  #rearrange column order and drop old product_action column
  df = df[['session_id_hash', 'sequence_action', 'timestamp', 'label']]

  #drop sequences shorter than 5 and longer than 155
  index_to_drop = []
  for id, row in df.iterrows():
    if len(row[1]) < 5 or len(row[1]) > 155:
      index_to_drop.append(id)
  df = df.drop(df.index[index_to_drop])

  #add exit event
  def add_exit(df):
    sequence_exit = []
    for id, row in df.iterrows():
      row[1] = [0] + row[1] + [6]
      sequence_exit.append(row[1])
    df['sequence_exit'] = sequence_exit
    return df
  add_exit(df)

  # add durations
  def delta(df):
    delta = []
    for t in df['timestamp']:
      v = [0] + list(np.diff([t[0]] + t))
      v = np.asarray(v)
      delta.append(v)
    df['delta'] = delta
    return df
  delta(df)
  
  #reset index
  df = df[['session_id_hash', 'sequence_exit', 'timestamp', 'delta', 'label']]
  df = df.reset_index()
  df.drop('index', inplace=True, axis=1)

  return df


In [ ]:
### Preprocess Dataframe
df2 = preprocess(df)
df2

In [ ]:
#plot average time on site
average_time_on_site = []
for row in df2['timestamp']:
  average_time_on_site.append(row[-1]-row[0])
q25, q75 = np.percentile(average_time_on_site,[.25,.75])
bin_width = 2*(q75 - q25)*len(average_time_on_site)**(-1/3)
average_time_on_site = np.asarray(average_time_on_site)
bins = round((average_time_on_site.max() - average_time_on_site.min())/bin_width)
print("Freedman–Diaconis number of bins:", bins)
plt.hist(average_time_on_site, density=True, bins=100)  # density=False would make counts
plt.ylabel('Frequency')
plt.xlabel('Time spent on the website');

In [ ]:
average_time_on_site = np.asarray(average_time_on_site)
average_time_on_site= average_time_on_site/1000

In [ ]:
average_duration = []
for row in df2['delta']:
  row = np.asarray(row[2:])
  mean = np.mean(row)
  if mean < 500:
    average_duration.append(mean)
average_duration = np.asarray(average_duration)
plt.hist(average_duration, density=False, bins=150, color = "skyblue", )  # density=False would make counts
plt.ylabel('counts')
plt.xlabel('average time spent on page');
plt.title("Average time between consecutive actions", size = 15, pad = 10)

In [ ]:
average_d = pd.DataFrame()
average_d['average duration'] = average_duration

In [ ]:
print(np.mean(average_duration))
print(np.std(average_duration))
print(max(average_duration))

In [ ]:
#Bar plot
view_c = 0
detail_c= 0
add_c= 0
remove_c= 0
click_c= 0
view_nc= 0
detail_nc= 0
add_nc= 0
remove_nc= 0
click_nc= 0
for id, row in df2.iterrows():
  if row[4]==0:
    for i in row[1]:
      if i == 1:
        view_c +=1
      elif i == 2:
        detail_c +=1
      elif i == 3:
        add_c +=1
      elif i == 4:
        remove_c +=1
      elif i == 5:
        click_c +=1
  elif row[4]==1:
    for i in row[1]:
      if i == 1:
        view_nc +=1
      elif i == 2:
        detail_nc +=1
      elif i == 3:
        add_nc +=1
      elif i == 4:
        remove_nc +=1
      elif i == 5:
        click_nc +=1
import numpy as np
import matplotlib.pyplot as plt

conversion = (view_c,detail_c,add_c,remove_c,click_c)
non_conversion = (view_nc,detail_nc,add_nc,remove_nc,click_nc)
ind = ('view','detail','add','remove','click') # the x locations for the groups
width = 0.35
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(ind, non_conversion, width, color='r')
ax.bar(ind, conversion, width,bottom=non_conversion , color='b')
ax.set_ylabel('Counts (in 100k)')
ax.set_title('Action events number by class',size = 15, pad = 10)
ax.set_xticks(ind, ('G1', 'G2', 'G3', 'G4', 'G5'))
ax.set_yticks(np.arange(0, 3000000, 300000))
ax.legend(labels=['Non-onversion', 'Conversion'])
plt.show()

In [ ]:
## histogram
length_sequences_C = []
length_sequences_NC = []

tot0 = df2[df2['label'] == 0].shape[0]
tot1 = df2[df2['label'] == 1].shape[0]

for item, y in zip(df2.sequence_exit, df2.label):
  if y == 0:
    length_sequences_C.append(len(item))
  else:
    length_sequences_NC.append(len(item))

plt.figure(figsize=(14,6))

plt.hist(length_sequences_NC, bins=150, alpha=0.4, label="Non- conversion sequences", density=True, color = "g")
plt.hist(length_sequences_C, bins=150, alpha=0.4, label="Conversion sequences", density=True, color = "orange")

plt.yscale('log')
plt.xlabel("Timesteps", size=14)
plt.ylabel("Probabilities", size=14)
plt.title("Histogram of Conversion and Non-conversion Trajectories ")
plt.legend(loc='upper right')
#plt.savefig("overlapping_histograms_with_matplotlib_Python.png")

In [ ]:
#plot class imbalance
pd.value_counts(df_1['label']).plot.bar()
plt.title('Label histogram')
plt.xlabel('Label')
plt.ylabel('Frequency')
df_1['label'].value_counts()

In [ ]:
from keras.preprocessing.sequence import pad_sequences
import sklearn
import numpy as np
from keras.utils import to_categorical

num_classes = 7
maxlen = 157

X1 = []
for sequence in df2['sequence_exit']:
  X1.append(to_categorical(sequence, num_classes = num_classes))
lol = X1
X1 = pad_sequences(X1, maxlen = maxlen, value = 0)
X2 = pad_sequences(df2['delta'], maxlen -1, value = -1)
X3 = df2['timestamp']

new_X = []
new_y = []
for x in X1:
  new_X.append(x[:-1])
  new_y.append(x[1:])

new_X = np.asarray(new_X)
new_y = np.asarray(new_y)


X = pd.DataFrame({'new_X': list(new_X), 'X2': list(X2), 'X3': list(X3), 'new_y': list(new_y)}, columns=['new_X', 'X2', 'X3', 'new_y'])

y = df2['label']

In [ ]:
#test train split
from sklearn.model_selection import train_test_split

X_train1, X_val1, y_train1, y_val1 = train_test_split(X, y, test_size=0.3, random_state=42, stratify = y)
X_val2, X_test2, y_val2, y_test2 = train_test_split(X_val1, y_val1, test_size=0.5, random_state=42, stratify = y_val1)

In [ ]:
#label sets

from numpy import where
from collections import Counter


y_train = pd.DataFrame()
y_train['new_y'] = X_train1['new_y']

y_train['y_label'] = y_train1

y_val = pd.DataFrame()
y_val['new_y'] = X_val2['new_y']

y_val['y_label'] = y_val2

y_test = pd.DataFrame()
y_test['new_y'] = X_test2['new_y']

y_test['y_label'] = y_test2

# summarize the new class distribution
counter = Counter(y_train)
print(counter)

print(len(X_train1))
print(len(X_val2))
print(len(X_test2))

print(len(y_train1))
print(len(y_val2))
print(len(y_test2))

In [ ]:
#reshape all data
X_train_new = np.stack(X_train1['new_X']).reshape((len(X_train1['new_X']), (maxlen -1 ) , num_classes)).astype(np.float32)
X_val_new = np.stack(X_val2['new_X']).reshape((len(X_val2['new_X']), (maxlen -1 ), num_classes)).astype(np.float32)
X_test_new = np.stack(X_test2['new_X']).reshape((len(X_test2['new_X']), (maxlen -1), num_classes)).astype(np.float32)
X_train_delta = np.stack(X_train1['X2']).reshape((len(X_train1['X2']), maxlen-1)).astype(np.float32)
X_val_delta = np.stack(X_val2['X2']).reshape((len(X_val2['X2']),maxlen -1)).astype(np.float32)
X_test_delta = np.stack(X_test2['X2']).reshape((len(X_test2['X2']),maxlen -1)).astype(np.float32)
X_train_time = np.asarray(X_train1['X3'])
X_val_time = np.asarray(X_val2['X3'])
X_test_time = np.asarray(X_test2['X3'])

y_train_label = y_train['y_label']
y_val_label = y_val['y_label']
y_test_label = y_test['y_label']

y_train_new = np.stack(y_train['new_y']).reshape((len(y_train['new_y']), (maxlen -1 ), num_classes)).astype(np.float32)
y_val_new = np.stack(y_val['new_y']).reshape((len(y_val['new_y']), (maxlen -1), num_classes)).astype(np.float32)
y_test_new = np.stack(y_test['new_y']).reshape((len(y_test['new_y']), (maxlen -1 ), num_classes)).astype(np.float32)


In [ ]:
print(X_train_new.shape)
print(X_val_new.shape)
print(X_test_new.shape)
print('---------------')
print(X_train_delta.shape)
print(X_val_delta.shape)
print(X_test_delta.shape)
print('---------------')
print(y_train_new.shape)
print(y_val_new.shape)
print(y_test_new.shape)

In [ ]:
#Discriminative specialised LSTM
from keras.layers import LSTM, Dense, Masking, Input, BatchNormalization
from keras import  Model

main_input = Input(shape=(maxlen -1, num_classes), name='main_input')
mask = Masking(mask_value = [0.,0.,0.,0.,0.,0.,0.], name = 'mask' )(main_input)
# train a 2-layer LSTM with one shared layer
l1 = LSTM(64, return_sequences=True)(mask)  # the shared layer
b1 = BatchNormalization()(l1)

l2_1 = LSTM(64, return_sequences=False)(b1)  # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)

binary_output = Dense(1, activation='sigmoid', name='binary_output')(b2_1)

model_SD = Model(inputs=[main_input], outputs=[binary_output])

model_SD.compile(loss= 'binary_crossentropy',optimizer = 'adam', metrics = ['acc'])

NameError: ignored

In [ ]:
#Fit model
batch_size = 64
epochs = 5
history_SD = model_SD.fit(X_train_new, y_train_label,
                         validation_data = (X_val_new, y_val_label),
                         verbose=1,
                         batch_size=batch_size,
                         epochs=epochs)

In [ ]:
# model predict
y_pred_binary_SD = model_SD.predict(X_test_new)

In [ ]:
#save model
model_SD.save('model_SD.h5')

In [ ]:
#load model
model_SD = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Thesis/model_SD.h5')

In [ ]:
#TIME generative specialised TIME LSTM
from keras.layers import LSTM, Dense, Masking, Input, BatchNormalization
from keras import  Model

main_input = Input(shape=(maxlen -1, num_classes), name='main_input')
mask1 = Masking(mask_value = 0, name = 'mask1' )(main_input)
# delta_ts: (batch_size, 1)
delta_ts = Input(shape=(maxlen -1, ), name='delta_ts')
mask2 = Masking(mask_value = -1, name = 'mask2' )(delta_ts)
# train a 2-layer LSTM with one shared layer
l1 = TLSTM_layer(64, return_sequence=True)(mask1,mask2)  # the shared layer
b1 = BatchNormalization()(l1)

l2_1 = TLSTM_layer(64, return_sequence=True)(b1,mask2)  # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)

act_output = Dense(num_classes, activation='softmax', name='act_output')(b2_1)

time_model_SG = Model(inputs=[main_input, delta_ts], outputs=[act_output])

time_model_SG.compile(loss= 'categorical_crossentropy',optimizer = 'adam', metrics = ['acc'])

In [ ]:
#Fit model
batch_size = 64
epochs = 5
history_time_SG = time_model_SG.fit([X_train_new, X_train_delta], y_train_new,
                         validation_data = ([X_val_new, X_val_delta], y_val_new),
                         verbose=1,
                         batch_size=batch_size,
                         epochs=epochs)

In [ ]:
y_pred_act_TIME_SG= time_model_SG.predict([X_test_new, X_test_delta])

In [ ]:
time_model_SG.save('time_model_SG.h5')

INFO:tensorflow:Assets written to: time_model_SG.tf/assets


In [ ]:
from src import TLSTM_layer
time_model_SG = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Thesis/time_model_SG.h5', custom_objects={'TLSTM_layer':TLSTM_layer})

In [ ]:
#TIME discriminative specialised TIME LSTM
from keras.layers import LSTM, Dense, Masking, Input, BatchNormalization, Lambda
from keras import  Model
from src import TLSTM_layer
main_input = Input(shape=(maxlen -1, num_classes), name='main_input')
mask1 = Masking(mask_value = 0, name = 'mask1' )(main_input)
# delta_ts: (batch_size, 1)
delta_ts = Input(shape=(maxlen -1, ), name='delta_ts')
mask2 = Masking(mask_value = -1, name = 'mask2' )(delta_ts)
# train a 2-layer LSTM with one shared layer
l1 = TLSTM_layer(64, return_sequence=True)(mask1,mask2)  # the shared layer
b1 = BatchNormalization()(l1)

l2_2a = TLSTM_layer(64, return_sequence=True)(b1, mask2)
l2_2 = Lambda(lambda x: x[:, -1, :])(l2_2a)
b2_1 = BatchNormalization()(l2_2)

binary_output = Dense(1, activation='sigmoid', name='act_output')(b2_1)

time_model_SD = Model(inputs=[main_input, delta_ts], outputs=[binary_output])

time_model_SD.compile(loss= 'binary_crossentropy',optimizer = 'adam', metrics = ['acc'])


In [ ]:
#Fit model
batch_size = 64
epochs = 5
history_time_SD = time_model_SD.fit([X_train_new, X_train_delta], y_train_label,
                         validation_data = ([X_val_new, X_val_delta], y_val_label),
                         verbose=1,
                         batch_size=batch_size,
                         epochs=epochs)

In [ ]:
# model predict
y_pred_binary_TIME_SD = time_model_SD.predict([X_test_new, X_test_delt])

In [ ]:
time_model_SD.save('time_model_SD.h5')

In [ ]:
time_model_SD = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Thesis/time_model_SD.h5',custom_objects={'TLSTM_layer':TLSTM_layer})

NameError: ignored

In [ ]:
#Generative specialised LSTM
from keras.layers import LSTM, Dense, Masking, Input, BatchNormalization
from keras import  Model

main_input = Input(shape=(maxlen -1, num_classes), name='main_input')
mask = Masking(mask_value = [0.,0.,0.,0.,0.,0.,0.], name = 'mask' )(main_input)
# train a 2-layer LSTM with one shared layer
l1 = LSTM(64, return_sequences=True)(mask)  # the shared layer
b1 = BatchNormalization()(l1)

l2_1 = LSTM(64, return_sequences=True)(b1)  # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)

act_output = Dense(num_classes, activation='softmax', name='act_output')(b2_1)

model_SG = Model(inputs=[main_input], outputs=[act_output])

model_SG.compile(loss= 'categorical_crossentropy',optimizer = 'adam', metrics = ['acc'])

In [ ]:
#Fit model
batch_size = 64
epochs = 5
history = model_SG.fit(X_train_new, y_train_new,
                         validation_data = (X_val_new, y_val_new),
                         verbose=1,
                         batch_size=batch_size,
                         epochs=epochs)

In [ ]:
# model predict
y_pred_act_SG = model_SG.predict(X_test_new)

In [ ]:
#save model
model_SG.save('model_SG.h5')

In [ ]:
model_SG = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Thesis/model_SG.h5')

In [ ]:
## Gen and discriminative LSTM
from keras.layers import LSTM, Dense, Masking, Input, BatchNormalization
from keras import  Model

main_input = Input(shape=(maxlen -1, num_classes), name='main_input')
mask = Masking(mask_value = [0.,0.,0.,0.,0.,0.,0.], name = 'mask' )(main_input)
# train a 2-layer LSTM with one shared layer
l1 = LSTM(64, return_sequences=True)(mask)  # the shared layer
b1 = BatchNormalization()(l1)

l2_1 = LSTM(64, return_sequences=True)(b1)  # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)

l2_2 = LSTM(64)(b1)  # the layer specialized in classifcation
b2_2 = BatchNormalization()(l2_2)

act_output = Dense(num_classes, activation='softmax', name='act_output')(b2_1)
binary_output = Dense(1, activation='sigmoid',name='binary_output')(b2_2)

model = Model(inputs=[main_input], outputs=[act_output, binary_output])
model.compile(loss={'act_output': 'categorical_crossentropy', 'binary_output': 'binary_crossentropy'},
                      optimizer = 'adam', metrics = ['acc'])

In [ ]:
#Fit model
batch_size = 64
epochs = 5
history = model.fit(X_train_new,{'act_output': y_train_new, 'binary_output': y_train_label},
                         validation_data = (X_val_new,{'act_output': y_val_new, 'binary_output': y_val_label}),
                         verbose=1,
                         batch_size=batch_size,
                         epochs=epochs)

In [ ]:
#Plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('LSTM loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# model predict
y_pred_act1, y_pred_binary1 = model.predict(X_test_new)

In [ ]:
#save model
model.save('vanilla.h5')

In [ ]:
#load model
model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/vanilla.h5')

In [ ]:
#discriminative report
import sklearn
print(sklearn.metrics.classification_report(y_test_label, t_y_pred_binary))

In [ ]:
# autoregressive report
print(sklearn.metrics.classification_report(y_test_new, y_pred_act))

In [ ]:
#Plot probability of exit for one trajectory:
def get_x_y(number):
  one = []
  for item in y_pred_act[number]:
    one.append(item[-1])

  item = [np.argmax(y, axis=None, out=None) for y in X_test_new[number]]
  item = np.ma.masked_equal(item, 0)
  item = item.compressed()
  lol = []
  for e in item:
    lol.append(e)
  x = []
  for i in range(len(lol)):
    x.append(i)

  y = one[-len(lol):]

  return x, y
x, y = get_x_y(9877)
plt.plot(x, y)
plt.show()

In [ ]:
## Time aware Gen and discriminative LSTM 
from keras.layers import LSTM, Dense, Masking, Input, BatchNormalization, Lambda
from keras import  Model

main_input = Input(shape = (maxlen - 1, num_classes), name = 'main_input')
mask1 = Masking(mask_value = 0, name = 'mask1' )(main_input)
# delta_ts: (batch_size, 1)
delta_ts = Input(shape=(maxlen -1, ), name='delta_ts')
mask2 = Masking(mask_value = -1, name = 'mask2' )(delta_ts)

l1 = TLSTM_layer(64, return_sequence = True)(mask1, mask2)

# batchnorm shared TLSTM layer output
b1 = keras.layers.BatchNormalization()(l1)

# the layer specialized in time prediction
l2_1 = TLSTM_layer(64, return_sequence = True)(b1, mask2)

# batchnorm activity prediction specialized LSTM
b2_1 = keras.layers.BatchNormalization()(l2_1)

# the layer specialized in binary classification
l2_2a = TLSTM_layer(64)(b1, mask2)
l2_2 = Lambda(lambda x: x[:, -1, :])(l2_2a)

# batchnorm time prediction specialized TLSTM output
b2_2 = keras.layers.BatchNormalization()(l2_2)

#d2_1 = keras.layers.Dropout(rate=hparams[HP_DROPOUT])(b2_1)
#d2_2 = keras.layers.Dropout(rate=hparams[HP_DROPOUT])(b2_2)

act_output = Dense(num_classes, activation = 'softmax', name='act_output')(b2_1)
binary_output = Dense(1, activation = 'sigmoid', name='binary_output')(b2_2)

# define model
time_model = Model(inputs=[main_input, delta_ts], outputs=[act_output, binary_output])

# compile model
time_model.compile(loss={'act_output': 'categorical_crossentropy', 'binary_output': 'binary_crossentropy'},
                      optimizer = 'adam', metrics = ['acc'])
time_model.summary()


In [ ]:
from IPython.display       import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(time_model, show_shapes = True, show_layer_names = True, dpi=65).create(prog='dot', format='svg'))

In [ ]:
#fit time model
batch_size = 64,
epochs = 5
history_time = time_model.fit([X_train_new, X_train_delta],
                          [y_train_new, y_train_label],
                         validation_data = ([X_val_new, X_val_delta],
                                             [y_val_new, y_val_label]),
                         verbose = 1,
                         batch_size = 64,
                         epochs = epochs)

In [ ]:
#Plot loss
plt.plot(history_time.history['loss'])
plt.plot(history_time.history['val_loss'])
plt.title('LSTM loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
y_pred_act, y_pred_binary = time_model.predict([X_test_new, X_test_delta])

In [ ]:
# save model
time_model.save('time_model_new.h5')

In [ ]:
#load model
time_model = tf.keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/Thesis/time_model_new.h5', custom_objects= {'TLSTM_layer':TLSTM_layer})

In [ ]:
#discriminative report
sklearn.metrics.classification_report(y_test_label, y_pred_binary)

In [ ]:
# autoregressive report
sklearn.metrics.classification_report(y_test_new, y_pred_act)

In [ ]:
#Plot probability of exit for one trajectory:
def get_x_y(number):
  one = []
  for item in y_pred_act_TIME_SG[number]:
    one.append(item[-1])

  item = [np.argmax(y, axis=None, out=None) for y in X_test_new[number]]
  item = np.ma.masked_equal(item, 0)
  item = item.compressed()
  lol = []
  for e in item:
    lol.append(e)
  x = []
  for i in range(len(lol)):
    x.append(i)

  y = one[-len(lol):]

  return x, y
x, y = get_x_y(14002)
plt.plot(x, y, label = 'exit risk' )
plt.title('Single task Time LSTM')
plt.xlabel("timesteps", size=14)
plt.ylabel("risk", size=14)
plt.legend(loc=0)
plt.show()

In [ ]:
#Recall, F1-score, AUC ROC
from sklearn.metrics import roc_auc_score
print('-----------')
print('AUC ROC')
print('Specialized LSTM')
print(roc_auc_score(y_test_label, y_pred_binary_SD))
print('------------')
print('Multi LSTM')
print(roc_auc_score(y_test_label, y_pred_binary1))
print('------------')
print('Multi Time LSTM')
print(roc_auc_score(y_test_label, y_pred_binary))

In [ ]:
#Perplexity
from keras import backend as K
def ppl_e(y_true, y_pred):
    return K.exp(K.mean(K.categorical_crossentropy(y_true, y_pred)))
print('-------------')
print('Time LSTM')
print(ppl_e(y_test_new,y_pred_act))
print('-------------')
print('Multi LSTM')
print(ppl_e(y_test_new,y_pred_act1))
print('--------------')
print('Specialized LSTM')
print(ppl_e(y_test_new,y_pred_act_SG))
print('--------------')
print('Specialized TIME LSTM')
print(ppl_e(y_test_new,y_pred_act_TIME_SG))

In [ ]:
#ROC CURVE MULTIPLR
from sklearn import metrics
plt.figure(0).clf()

fpr, tpr, thresh = metrics.roc_curve(y_test_label, y_pred_binary_SD)
auc = metrics.roc_auc_score(y_test_label, y_pred_binary_SD)
plt.plot(fpr,tpr,label="LSTM_S, auc="+str(auc))

fpr, tpr, thresh = metrics.roc_curve(y_test_label,y_pred_binary1)
auc = metrics.roc_auc_score(y_test_label, y_pred_binary1)
plt.plot(fpr,tpr,label="Multi-task LSTM, auc="+str(auc))

fpr, tpr, thresh = metrics.roc_curve(y_test_label, y_pred_binary)
auc = metrics.roc_auc_score(y_test_label, y_pred_binary)
plt.plot(fpr,tpr,label="Multi_task T-LSTM, auc="+str(auc))
plt.xlabel("FPR", size=14)
plt.ylabel("TPR", size=14)
plt.title("ROC curves")
plt.plot([0,1], [0,1], linestyle='--', label='No Skill, auc =0.5')
plt.legend(loc=0)

In [ ]:
#POst-Processsing
#random choice

def make_y_sequences(y_pred_act):
  random_pred = []
  real_pred = []
  for i in range(len(y_pred_act)):
    item = [np.argmax(y, axis=None, out=None) for y in y_test_new[i]]
    item = np.ma.masked_equal(item, 0)
    item = item.compressed()
    real_pred.append(item)
    l = len(item)
    ns = []
    for r in y_pred_act[i][-l:]:
      wow =np.random.choice(r, p=r)
      for i in range(len(r)):
        if r[i] == wow:
          ns.append(i)
    random_pred.append(ns)
  return random_pred, real_pred
random_pred, real_pred = make_y_sequences(y_pred_act)
random_pred1, real_pred = make_y_sequences(y_pred_act1)
random_pred_SG, real_pred = make_y_sequences(y_pred_act_SG)
random_pred_TIME_SG, real_pred = make_y_sequences(y_pred_act_TIME_SG)

In [ ]:
#DL distance from https://stackabuse.com/levenshtein-distance-and-text-similarity-in-python/
import numpy as np

def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    return (matrix[size_x - 1, size_y - 1])

In [ ]:
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, MaxPooling2D
from keras.layers.convolutional import Conv2D 
from keras.layers.recurrent import SimpleRNN, LSTM, GRU 
from keras.utils import np_utils
from keras import backend as K

from distutils.version import LooseVersion as LV
from keras import __version__

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from keras.datasets import mnist, fashion_mnist, imdb

from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print('Using Keras version:', __version__, 'backend:', K.backend())
assert(LV(__version__) >= LV("2.0.0"))

Using Keras version: 2.4.3 backend: tensorflow


In [ ]:
if K.backend() == "tensorflow":
    import tensorflow as tf
    device_name = tf.test.gpu_device_name()
    if device_name == '':
        device_name = "None"
    print('Using TensorFlow version:', tf.__version__, ', GPU:', device_name)

Using TensorFlow version: 2.4.1 , GPU: /device:GPU:0


In [ ]:
#distance similarity array
distances = []
distances1 = []
distances_SG = []
distances_TIME_SG = []
for ps, rs in zip(random_pred, real_pred):
    distances.append(levenshtein(ps, rs))
for ps, rs in zip(random_pred1, real_pred):
    distances1.append(levenshtein(ps, rs))
for ps, rs in zip(random_pred_SG, real_pred):
    distances_SG.append(levenshtein(ps, rs))
for ps, rs in zip(random_pred_TIME_SG, real_pred):
    distances_TIME_SG.append(levenshtein(ps, rs))

In [ ]:
# DL mean
print('DL mean')
print('---------')
print('Multi-task TIME lstm')
print(np.mean(distances))
print('---------')
print('Multi-task lstm')
print(np.mean(distances1))
print('---------')
print('Single-task lstm')
print(np.mean(distances_SG))
print('---------')
print('Single-task TIME lstm')
print(np.mean(distances_TIME_SG))


In [ ]:
#---------------END OF CODE---------------------#